In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import CrossEntropyLoss
from sklearn.model_selection import KFold

## Pre-Process Train Data 

In [ ]:
train_dataset = pd.read_csv("../Dataset_DAY1/Data/train_set.csv", delimiter=';')

In [ ]:
train_dataset.head(30)

In [ ]:
## drop features
def Drop_unneed_columns(dataset):
    cols= ['application_ID', 'company_ID', 'decision_date']
    dataset= dataset.drop(columns=cols)
    return dataset

In [ ]:
def Nan_values(dataset):
    column_names = dataset.columns.tolist()
    drop_columns = []
    for name in column_names:
        nan_count = dataset[name].isna().sum()
        print(f"column {name}: {nan_count}")
        if (nan_count/28000) > 0.5:
            print(f"Number of NaN values in column '{name}': {nan_count}")
            drop_columns.append(name)
    return drop_columns

In [ ]:
def Replace_cate_to_value(column_name, dataset):
    # Extract categories

    # Extract unique category names from the column
    unique_categories = dataset[column_name].unique()

    # convert 'numpy.ndarray' in to a python list
    l = unique_categories.tolist()
    
    if 'MISSING' in l:
        l.remove('MISSING')
        l.sort(reverse=True)

    print(l)
    
    dic = { l[i]:i+1 for i in range(0, len(l))}

    # Replace values in the column based on the dictionary mapping
    dataset[column_name] = dataset[column_name].replace(dic)
    return dic, dataset

In [ ]:
def Category_values(dataset):
    column_names = ['industry_sector','region', 'geo_area','external_score_ver03', 'province','juridical_form']
    dic = {}
    for column_name in column_names:
        category_dic, dataset = Replace_cate_to_value(column_name, dataset)
        dic[column_name] = category_dic
    return dic, dataset

In [ ]:
def Replace_bool_toNumbers(dataset):
    dataset['cr_available'] = [int(dataset['cr_available'][i]) for i in range(len(dataset['cr_available']))]
    dataset['cr_available']
    return dataset

In [ ]:
def mean_var03(dataset):
    s0, s1, c0, c1 = 0,0,0,0
    # unique_labels = dataset['target'].unique()
    for index, row in dataset.iterrows():
        if row['external_score_ver03'] != 'MISSING':
            if row['target'] == 0:
                s0 += row['external_score_ver03']
                c0 +=1
            elif row['target'] == 1:
                s1 +=  row['external_score_ver03']
                c1 += 1

    m0 = round(s0/c0)
    m1 = round(s1/c1)
    print(m0)
    print(m1)
    return m0,m1

In [ ]:
def Replace_missing(dataset, m0, m1):
    # Assuming df is your DataFrame and 'column_to_change' is the column you want to change
    # 'condition_column' is the column based on which you want to change the content
    dataset.loc[(dataset['target'] == 1) & (dataset['external_score_ver03'] == 'MISSING'), 'external_score_ver03'] = m1
    dataset.loc[(dataset['target'] == 0) & (dataset['external_score_ver03'] == 'MISSING'), 'external_score_ver03'] = m0
    dataset['external_score_ver03']

    # For example, if you want to change the content of 'column_to_change' to 'new_value' where 'condition_column' is True
    # Replace 'new_value', 'column_to_change', and 'condition_column' with your actual values
    return dataset

In [ ]:
# Drop columns 
train_dataset = Drop_unneed_columns(train_dataset)
drop_columns = Nan_values(train_dataset)
train_dataset = train_dataset.drop(columns=drop_columns)

In [ ]:
# replace bool values to numerical ones 
category_dics, train_dataset = Category_values(train_dataset)
train_dataset = Replace_bool_toNumbers(train_dataset)

In [ ]:
# v03 column with missing values 
m0, m1= mean_var03(train_dataset)
train_dataset = Replace_missing(train_dataset, m0, m1)

In [ ]:
def normalized_data(dataset):
    # Replace commas with periods in all columns
    dataset = dataset.replace(',', '.', regex=True)
    print(dataset.dtypes)
    dataset = dataset.astype('float32')

    # check if the dataset has any nan value
    has_nan_values = dataset.isna().any().any()

    if has_nan_values:
        print("DataFrame contains NaN values.")
    else:
        print("DataFrame does not contain any NaN values.")

    return dataset

In [ ]:
train_dataset = normalized_data(train_dataset)
train_dataset.head()

For the Model

In [ ]:
y = train_dataset['days_to_default'].to_numpy() # labels
X = train_dataset.drop(columns=['days_to_default']).to_numpy()

## <font color="yellow"> SVM Training 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

# Assuming Y contains the target variable for regression

# Standardize the features (mean=0 and variance=1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(train_dataset.drop(columns='days_to_default'))


# Standardize the target variable Y
Y = train_dataset['days_to_default']
scaler_Y = StandardScaler()
Y_scaled = scaler_Y.fit_transform(Y.values.reshape(-1, 1))  # Reshape Y to be a 2D array for StandardScaler


# Create PCA object
pca = PCA(n_components=20)  # Specify the number of components (desired dimensionality)

# Fit PCA to the standardized data and transform the data
X_pca = pca.fit_transform(X_scaled)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X_pca, Y, test_size=0.1, random_state=2)

# Create SVR (Support Vector Regression) model
regressor = SVR(C=0.1, kernel='linear', gamma='scale')

# Fit the model on the training data
regressor.fit(X_train, Y_train)

# Predict on the testing data
Y_pred = regressor.predict(X_test)

# Calculate Mean Squared Error (MSE) as a metric
mse = mean_squared_error(Y_test, Y_pred)
print("Mean Squared Error:", mse)

## <font color="yellow"> Random Forest Training